# 트리 앙상블

> 대표적인 앙상블 학습인 랜덥 포레스트 모델에 대해서 알아본다.

* 사용한 모델 : 

## 랜덤 포레스트 (Random Forest)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv("https://bit.ly/wine_csv_data")
data = wine[["alcohol", "sugar", "pH"]].to_numpy()
target = wine["class"].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9973541965122431 0.8905151032797809


In [3]:
# 특성 중요도
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
# OOB(Out Of Bag) 샘플
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 액스트라 트리 (Extra Tree)

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅 (Gradient boosting)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.8881086892152563 0.8720430147331015


## 히스토그램 기반 그레이디언트 부스팅 (Histogram-based Gradient boosting)

In [8]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9321723946453317 0.8801241948619236


In [9]:
# 훈련 세트에서 특성 중요도 계산
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result)
print(result["importances_mean"])

{'importances_mean': array([0.08876275, 0.23438522, 0.08027708]), 'importances_std': array([0.00382333, 0.00401363, 0.00477012]), 'importances': array([[0.08793535, 0.08350972, 0.08908986, 0.08312488, 0.09274581,
        0.08755051, 0.08601116, 0.09601693, 0.09082163, 0.09082163],
       [0.22782374, 0.23590533, 0.23936887, 0.23436598, 0.23725226,
        0.23436598, 0.23359631, 0.23398114, 0.23994612, 0.22724649],
       [0.08581874, 0.08601116, 0.08062344, 0.07504329, 0.08427939,
        0.07792957, 0.07234943, 0.07465846, 0.08139311, 0.08466423]])}
[0.08876275 0.23438522 0.08027708]


In [10]:
# 테스트 세트에서 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [11]:
# 테스트 세트에서의 성능
print(hgb.score(test_input, test_target))

0.8723076923076923


## XGBoost

In [15]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method="hist", random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9555033709953124 0.8799326275264677
